In [ ]:
# @title Clicar Play Para Rodar Codigo
!pip install --quiet pandas openpyxl plotly openai ipywidgets matplotlib fpdf tqdm
import pandas as pd
import matplotlib.pyplot as plt
import io, os, re
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from google.colab import files
from fpdf import FPDF
from datetime import datetime
from tqdm.notebook import tqdm
uploaded = files.upload()
csv_path = next(iter(uploaded))
img_path = "/content/csn logo.png"
api_key = input("Digite sua chave da OpenAI (sk-...): ").strip()
client = OpenAI(api_key=api_key)

def ler_csv_br(filepath):
    tentativas = [
        {"sep": ";", "encoding": "utf-8"},
        {"sep": ";", "encoding": "ISO-8859-1"},
        {"sep": ",", "encoding": "utf-8"},
        {"sep": ",", "encoding": "ISO-8859-1"},
    ]
    for tentativa in tentativas:
        try:
            df = pd.read_csv(filepath, sep=tentativa["sep"], encoding=tentativa["encoding"], skiprows=3)
            df.columns = df.columns.str.strip()
            return df
        except Exception:
            continue
    raise ValueError("❌ Não foi possível ler o CSV.")

df = ler_csv_br(csv_path)
colunas_numericas = ["Ativa C (kWh)", "Ativa G (kWh)", "Reativa C (kVArh)", "Reativa G (kVArh)"]
for col in colunas_numericas:
    df[col] = df[col].astype(str).str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    df[col] = pd.to_numeric(df[col], errors="coerce")
df["Data - Base diária"] = pd.to_datetime(df["Data - Base diária"], dayfirst=True)

agentes = sorted(df["Agente"].dropna().unique())
dropdown_agente = widgets.Dropdown(options=agentes, description="Agente:")
dropdown_grandeza = widgets.Dropdown(options=[], description="Grandeza:")
botao_gerar = widgets.Button(description="Gerar Análise", button_style="success")
botao_pdf = widgets.Button(description="📄 Análise Geral", button_style="info")

def formatar_analise(texto):
    texto = re.sub(r"\*\*(.+?)\*\*", r"\1", texto)
    texto = re.sub(r"###\s(.+)", r"<<TITULO>>\1<<FIM>>", texto)
    texto = re.sub(r"[\u2022\u25CF\u25A0]", "-", texto)
    texto = texto.encode('latin-1', 'ignore').decode('latin-1')
    return texto

def gerar_analise_ai(texto_csv, nome_grandeza):
    prompt = f"""
Você é um analista de dados especializado em consumo de energia em ambientes industriais. Com base na série temporal abaixo referente à grandeza '{nome_grandeza}', forneça uma análise executiva da operação da planta durante o período analisado.
Evite repetir números exatos. Em vez disso, descreva:
- Tendências predominantes (alta, queda, estabilidade, flutuação)
- Possíveis motivos técnicos ou operacionais para os comportamentos observados
- Impactos potenciais na eficiência ou no custo da planta
- Sugestões ou hipóteses baseadas no padrão de consumo
Dados analisados:
{texto_csv}
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "Você é um especialista em análise energética."},
                  {"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=1000,
    )
    return formatar_analise(response.choices[0].message.content)

def atualizar_grandezas(*args):
    agente = dropdown_agente.value
    dropdown_grandeza.options = [col for col in colunas_numericas if df[df["Agente"] == agente][col].fillna(0).sum() > 0]
dropdown_agente.observe(atualizar_grandezas, names="value")
atualizar_grandezas()

def ao_clicar_botao(b):
    clear_output(wait=True)
    display(widgets.HBox([dropdown_agente, dropdown_grandeza]), widgets.HBox([botao_gerar, botao_pdf]))
    agente = dropdown_agente.value
    grandeza = dropdown_grandeza.value
    dados = df[df["Agente"] == agente][["Data - Base diária", grandeza]].dropna().groupby("Data - Base diária").sum().reset_index()
    if dados.empty or dados[grandeza].sum() == 0:
        display(Markdown("⚠️ Nenhum dado válido encontrado."))
        return
    plt.figure(figsize=(10, 5))
    plt.plot(dados["Data - Base diária"], dados[grandeza], marker='o')
    plt.title(f"{grandeza} — {agente}")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    texto_csv = dados.head(14).to_csv(index=False)
    analise = gerar_analise_ai(texto_csv, grandeza)
    display(Markdown(f"### 🧠 **Análise gerada por IA:**\n\n{analise}"))

class PDFCustom(FPDF):
    def add_titulo(self, titulo):
        self.set_font("Arial", "BU", 12)
        self.multi_cell(0, 8, titulo)
        self.ln(2)
    def add_texto(self, texto):
        self.set_font("Arial", "", 10)
        self.multi_cell(0, 5, texto)
        self.ln(2)

def gerar_pdf_completo(b):
    clear_output(wait=True)
    display(Markdown("⏳ Gerando relatório..."))
    pdf = PDFCustom()
    pdf.add_page()
    pdf.image(img_path, x=150, y=10, w=50)
    pdf.set_font("Arial", "B", 16)
    pdf.set_xy(10, 10)
    pdf.multi_cell(100, 10, "Relatório Geral de Análise de Consumo", align='L')
    pdf.set_font("Arial", "", 12)
    pdf.set_xy(10, 30)
    pdf.cell(0, 10, f"Gerado em: {datetime.today().strftime('%d/%m/%Y')}")

    pares_validos = [(ag, gr) for ag in df["Agente"].dropna().unique()
                     for gr in colunas_numericas if df[df["Agente"] == ag][gr].fillna(0).sum() > 0]
    for agente, grandeza in tqdm(pares_validos):
        dados = df[df["Agente"] == agente][["Data - Base diária", grandeza]].dropna().groupby("Data - Base diária").sum().reset_index()
        img_temp = f"/tmp/{agente}_{grandeza}.png".replace(" ", "_")
        plt.figure(figsize=(6, 3))
        plt.plot(dados["Data - Base diária"], dados[grandeza], marker='o')
        plt.title(f"{grandeza} - {agente}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(img_temp)
        plt.close()
        texto_csv = dados.head(14).to_csv(index=False)
        analise = gerar_analise_ai(texto_csv, grandeza)

        pdf.add_page()
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, f"{agente} - {grandeza}", ln=True)
        pdf.image(img_temp, x=10, y=25, w=190)
        pdf.ln(110)

        for linha in analise.split("\n"):
            if linha.startswith("<<TITULO>>"):
                titulo = linha.replace("<<TITULO>>", "").replace("<<FIM>>", "").strip()
                pdf.add_titulo(titulo)
            else:
                pdf.add_texto(linha)

    pdf.output("Relatorio_Analise_Completo.pdf")
    display(Markdown("✅ **Relatório final gerado com sucesso:** `Relatorio_Analise_Completo.pdf`"))

botao_gerar.on_click(ao_clicar_botao)
botao_pdf.on_click(gerar_pdf_completo)
display(widgets.HBox([dropdown_agente, dropdown_grandeza]), widgets.HBox([botao_gerar, botao_pdf]))


